<a href="https://colab.research.google.com/github/ManavMoitra/basicml/blob/master/Monkey_Breed_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.3.0

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.keras.applications import MobileNet
img_rows,img_cols=224,224
with tf.device('/GPU:0'):

    MobileNet=MobileNet(weights='imagenet',
                        include_top=False,
                        input_shape=(img_rows,img_cols,3))
    for layer in MobileNet.layers:
        layer.trainable=False
    for (i,layer) in enumerate(MobileNet.layers):
        print(str(i)+" "+layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [5]:
def AddTopModelMobileNet(bottommodel,num_classes):
    topmodel=bottommodel.output
    topmodel=GlobalAveragePooling2D()(topmodel)
    topmodel=Dense(1024,activation='relu')(topmodel)
    topmodel=Dense(1024,activation='relu')(topmodel)
    topmodel=Dense(512,activation='relu')(topmodel)
    topmodel=Dense(num_classes,activation='softmax')(topmodel)
    return topmodel
    

In [6]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout,Activation,Flatten,MaxPooling2D,ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization

In [7]:
with tf.device('/GPU:0'):

    num_classes=10
    FC_Head=AddTopModelMobileNet(MobileNet,num_classes)
    model=Model(inputs=MobileNet.inputs,outputs=FC_Head)
    print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_dir=r'/content/drive/My Drive/monkey_species/training/training'
validation_data_dir=r'/content/drive/My Drive/monkey_species/validation/validation'

In [9]:
train_datagen=ImageDataGenerator(
                                 rescale=1./255,
                                 rotation_range=45,
                                 width_shift_range=0.3,
                                 height_shift_range=0.3,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
validation_datagen=ImageDataGenerator(rescale=1./255)

batch_size=32


train_generator=train_datagen.flow_from_directory(
                                                  train_data_dir,
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical'
                                                  )

validation_generator=validation_datagen.flow_from_directory(
                                                  validation_data_dir,
                                                  target_size=(img_rows,img_cols),
                                                  batch_size=batch_size,
                                                  class_mode='categorical'
                                                  )

Found 1110 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [10]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint=ModelCheckpoint(r"/content/drive/My Drive/monkey_species/modelcheckpoint",
                           monitor='val_loss',
                           mode='min',
                           save_best_only=True,
                           verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=3,
                        verbose=1,
                        restore_best_weights=True)

callbacks=[earlystop,checkpoint]

In [11]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [12]:
with tf.device('/GPU:0'):

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(lr=0.001),
                  metrics=['Accuracy'])

In [13]:
train_samples=1098
validation_samples=272
epochs=5
batch_size=16

In [14]:
with tf.device('/GPU:0'):

    history=model.fit(
                      train_generator,
                      steps_per_epoch=train_samples//batch_size,
                      epochs=epochs,
                      callbacks=callbacks,
                      validation_data=validation_generator,
                      validation_steps=validation_samples//batch_size)

Epoch 1/5
35/68 [==============>...............] - ETA: 1:22 - loss: 2.5192 - accuracy: 0.0000e+00WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 340 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: val_loss improved from inf to 1.30460, saving model to /content/drive/My Drive/monkey_species/modelcheckpoint
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/monkey_species/modelcheckpoint/assets
35/68 [==============>...............] - 300s 9s/step - loss: 2.5192 - accuracy: 0.0000e+00 - val_loss: 1.3046 - val_accuracy: 0.0000e+00
